## sentiment analysis basic model trying
Here we use pytorch and torchtext

In [2]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch 
from torchtext.legacy import data

SEED = 32
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Text = data.Field(tokenize = 'spacy' , tokenizer_language = 'en_core_web_sm')
Label = data.LabelField(dtype = torch.float)

In [7]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(Text, Label)

downloading aclImdb_v1.tar.gz


C:\Users\luzheng\ML_trainning\sentiment_analysis\Emotional_Analysis\Clock_In\.data\imdb\aclImdb_v1.tar.gz: 100%|█| 84.1


In [10]:
print('the len of train and test was :{} , {}'.format(len(train_data) , len(test_data)))

the len of train and test was :25000 , 25000


In [14]:
vars(train_data.examples[0]).keys()

dict_keys(['text', 'label'])

In [16]:
# show the whole sentence so i better know the structure of the data
' '.join(vars(train_data.examples[0])['text'])

'Bromwell High is a cartoon comedy . It ran at the same time as some other programs about school life , such as " Teachers " . My 35 years in the teaching profession lead me to believe that Bromwell High \'s satire is much closer to reality than is " Teachers " . The scramble to survive financially , the insightful students who can see right through their pathetic teachers \' pomp , the pettiness of the whole situation , all remind me of the schools I knew and their students . When I saw the episode in which a student repeatedly tried to burn down the school , I immediately recalled ......... at .......... High . A classic line : INSPECTOR : I \'m here to sack one of your teachers . STUDENT : Welcome to Bromwell High . I expect that many adults of my age think that Bromwell High is far fetched . What a pity that it is n\'t !'

In [17]:
# see the label 
vars(train_data.examples[0])['label']

'pos'

now we see the data we get by using vars()function could return a dict

In [19]:
import random

train_data, valid_data = train_data.split(split_ratio=0.8 , random_state = random.seed(SEED))
print('the len of train and valid was ；{},{}'.format(len(train_data) , len(valid_data)))

the len of train and valid was ；20000,5000


In [22]:
# we use the one_hot encoding by limit the length of the sentence sequence so that the we won't
#use much when train the model .
MAX_VOCAB_SIZE = 25000

Text.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
Label.build_vocab(train_data)


In [24]:
print(f"Unique tokens in TEXT vocabulary: {len(Text.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(Label.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [43]:
# import pandas as pd
# # dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])
# vars(Text.vocab).keys()
# frame_vocab = pd.DataFrame(dict(vars(Text.vocab)['freqs']))
# frame_vocab.head()

In [44]:
{k:v for k ,v in sorted(dict(vars(Text.vocab)['freqs']).items( ) , key = lambda x:x[1] , reverse = True)}

{'the': 231965,
 ',': 220100,
 '.': 189569,
 'a': 125667,
 'and': 125144,
 'of': 115567,
 'to': 107166,
 'is': 87226,
 'in': 70396,
 'I': 61765,
 'it': 61333,
 'that': 56479,
 '"': 50441,
 "'s": 49539,
 'this': 48321,
 '-': 43069,
 '/><br': 40882,
 'was': 40198,
 'as': 34893,
 'with': 34517,
 'movie': 34132,
 'for': 33760,
 'film': 30993,
 'The': 29989,
 'but': 28084,
 '(': 26711,
 'on': 26447,
 "n't": 26317,
 ')': 26242,
 'you': 24267,
 'are': 23890,
 'not': 23042,
 'his': 22560,
 'have': 22535,
 'be': 21289,
 'he': 19971,
 'one': 19397,
 '!': 17685,
 'by': 17456,
 'at': 17444,
 'all': 17093,
 'an': 16714,
 'who': 16495,
 'from': 15668,
 'they': 15655,
 'like': 15482,
 'her': 14391,
 'so': 14148,
 'about': 13725,
 'or': 13706,
 'has': 13630,
 "'": 13566,
 'It': 13406,
 'out': 13310,
 'just': 13291,
 'do': 12558,
 '?': 12027,
 'some': 11599,
 'good': 11544,
 'more': 11109,
 'very': 10700,
 'would': 10614,
 'up': 10478,
 'what': 10274,
 'This': 10107,
 'there': 9928,
 'time': 9669,
 'ca

我们将使用一个“BucketIterator”，它是一种特殊类型的迭代器，它将返回一批示例，其中每个样本的长度差不多，从而最小化每个样本的padding数。

In [48]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)


In [49]:
import torch.nn as nn 

class RNN(nn.Module):
    def __init__(self , input_dim , hidden_dim , embedding_dim , output_dim):
        super().__init__()
        self.embedding =nn.Embedding(input_dim , embedding_dim )
        self.rnn = nn.RNN(embedding_dim  , hidden_dim)
        self.fc = nn.Linear(hidden_dim , output_dim)
        
    def forward(self , text):
        # text = [text len , batch size]
        embedded = self.embedding(text)
        output , hidden = self.rnn(embedded)
        assert torch.equal(output[-1,: ,:] , hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

In [57]:
MAX_VOCAB_SIZE = 25000

Text.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
Label.build_vocab(train_data)

INPUT_DIM = len(Text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [58]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,436,413 trainable parameters


In [59]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [60]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [61]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [62]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [63]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 5m 38s
	Train Loss: 0.695 | Train Acc: 50.39%
	 Val. Loss: 0.695 |  Val. Acc: 50.65%
Epoch: 02 | Epoch Time: 5m 53s
	Train Loss: 0.693 | Train Acc: 50.08%
	 Val. Loss: 0.695 |  Val. Acc: 50.61%
Epoch: 03 | Epoch Time: 6m 1s
	Train Loss: 0.694 | Train Acc: 50.25%
	 Val. Loss: 0.695 |  Val. Acc: 50.51%
Epoch: 04 | Epoch Time: 5m 55s
	Train Loss: 0.694 | Train Acc: 50.00%
	 Val. Loss: 0.695 |  Val. Acc: 50.55%
Epoch: 05 | Epoch Time: 5m 54s
	Train Loss: 0.693 | Train Acc: 49.62%
	 Val. Loss: 0.695 |  Val. Acc: 50.53%
